# Adtracking Fraud Detection

by Kyle O'Brien, Catherine Lee, Amit Saxena 

Data is in "data/" and includes the training and testing csv files.
Evaluation metrics will include looking at R^2 and a confusion matrix. Other things to try, CNN (Resnet, inceptionv3)

In [1]:
import numpy as np         # linear algebra
import sklearn as sk       # machine learning
import pandas as pd        # reading in data files, data cleaning
import matplotlib.pyplot as plt   # for plotting
import seaborn as sns      # visualization tool
import tensorflow as tf
import keras

Using TensorFlow backend.


Train_sample: 100,000 randomly-selected rows of training data (because the full training data takes too long)

In [8]:
#load data in 

data = pd.read_csv("data/equalized_train.csv")

data.head()


,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,83252,3,1,16,379,2017-11-06 15:42:19,NaN,0
1,106590,3,1,25,379,2017-11-06 15:43:23,NaN,0
2,147164,14,1,28,134,2017-11-06 16:00:00,NaN,0
3,39782,2,1,10,205,2017-11-06 16:00:00,NaN,0
4,121646,23,1,13,153,2017-11-06 16:00:00,NaN,0


In [3]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
ip                 100000 non-null int64
app                100000 non-null int64
device             100000 non-null int64
os                 100000 non-null int64
channel            100000 non-null int64
click_time         100000 non-null object
attributed_time    227 non-null object
is_attributed      100000 non-null int64
dtypes: int64(6), object(2)
memory usage: 6.1+ MB


Dropping attributed_time because many null values

# Feature Engineering for Date and Time

In [4]:
#separate click_time into multiple columns
#data['click_time'].split(" ")


# Understanding our Dataset

In [5]:
#data = data.drop(columns=['attributed_time'])
#data.info()

In [6]:
data['is_attributed'].describe()

count    100000.000000
mean          0.002270
std           0.047591
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: is_attributed, dtype: float64

In [6]:
#plotx = data['attributed_time']
#ploty = data['is_attributed']

#plt.scatter(plotx,ploty)
import pandas
class_counts = data.groupby('is_attributed').size()
print(class_counts)

is_attributed
0    99773
1      227
dtype: int64


This confirms that we have a major imbalance in our data. This is incentivising our model to guess 0 for overthing and it would still receive a 99% accuracy

# Testing with Logistic Regression

In [9]:
X = data[['app', 'device', 'os', 'channel']]
y = data['is_attributed']

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [11]:
X_train.head()

,app,device,os,channel
22243,2,1,20,477
58933,105,1,17,451
50574,18,3032,607,107
63812,11,1,13,325
52847,116,1,41,101


In [12]:
y_train.head()

22243    0
58933    1
50574    0
63812    1
52847    1
Name: is_attributed, dtype: int64

In [13]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [14]:
logreg.fit(X_train, y_train)


/root/.local/share/virtualenvs/fraud_detection-yH1T5P0u/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [15]:
predictions = logreg.predict(X_test)

In [14]:
#from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(y_test,predictions)


Seems a little strange that we would get such high accuracy. This might have to do with the fact that we are using R^2 to measure the goodness of our model. There might be an imbalance in the test set of which clicks are 0 or 1, so we should try precision and recall. (Longterm with full training data, try to keep an equal number of classes)

In [15]:
from sklearn import metrics
print("avg precision-recall score:", metrics.average_precision_score(X_test,y_test))

ValueError: multiclass-multioutput format is not supported

In [16]:
score = logreg.score(X_test, y_test)
print(score)

0.6349822355834928


# Random forest

# Support Vector Machine

In [17]:
from sklearn.svm import LinearSVC 

In [18]:
lSVC = LinearSVC()

In [19]:
lSVC.fit(X_train,y_train)
print("accuracy:", lSVC.score(X_test,y_test))

accuracy: 0.6373872642798579


/root/.local/share/virtualenvs/fraud_detection-yH1T5P0u/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


# Decision Tree

In [20]:
from sklearn import tree
dtc = tree.DecisionTreeClassifier()

dtc.fit(X_train,y_train)
print("Decision Tree Accuracy:", dtc.score(X_test,y_test))

Decision Tree Accuracy: 0.9351188849412407


# Neural Network

In [ ]:
model = Sequential()
model.add(Dense(500, activation='sigmoid', input_dim = 13))
model.add(Dense(400,activation='sigmoid'))
model.add(Dense(200, activation='sigmoid'))

model.compile(optimizer = 'rmsprop',     
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

# XGBoost

# Ensemble Learning